In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

mnist = mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Normalize the pixels in 0.0~1.0 float
train_images, test_images = train_images / 255.0, test_images / 255.0

print(train_images.shape)
print(train_labels.shape)

(60000, 28, 28, 1)
(60000, 10)


In [2]:
# Define a model
class mnist_lenet(tf.keras.Model):
    def __init__(self):
        super(mnist_lenet, self).__init__()
        self.conv1 = layers.Conv2D(filters=10, kernel_size=[3,3], input_shape = (28,28,1), activation= 'relu')
        self.pool1 = layers.MaxPooling2D(2, 2)
        self.conv2 = layers.Conv2D(filters=20, kernel_size=[3,3], activation= 'relu')
        self.pool2 = layers.MaxPooling2D(2, 2)
        self.conv3 = layers.Conv2D(filters=30, kernel_size=[3,3], activation= 'relu')
        self.flat = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(10, activation='softmax')


    def call(self, x):
        net = self.conv1(x)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense2(net)
        return net

In [3]:
# Train & save model in frozen(.pb) format.

my_model = mnist_lenet()
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
my_model.fit(train_images, train_labels, batch_size=1, epochs=5, verbose=1)
my_model.summary()
my_model.save('mnist_lenet')

2024-10-29 14:45:42.634307: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-10-29 14:45:42.635715: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 14:45:42.637660: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-10-29 14:45:43.168106: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-10-29 14:45:43.170877: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2496000000 Hz


Epoch 1/5
60000/60000 [==============================] - 486s 8ms/step - loss: 0.2310 - accuracy: 0.9259
Epoch 2/5
60000/60000 [==============================] - 336s 6ms/step - loss: 0.0745 - accuracy: 0.9796
Epoch 3/5
60000/60000 [==============================] - 258s 4ms/step - loss: 0.0682 - accuracy: 0.9806
Epoch 4/5
60000/60000 [==============================] - 252s 4ms/step - loss: 0.0692 - accuracy: 0.9824
Epoch 5/5
60000/60000 [==============================] - 218s 4ms/step - loss: 0.0667 - accuracy: 0.9831
Model: "mnist_lenet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  100       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  182

2024-10-29 15:11:32.885481: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mnist_lenet/assets


In [4]:
# Load model and convert it for TensorFlow Lite (.tflite format)

import tensorflow as tf
import numpy as np

model_path = 'mnist_lenet'
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()


2024-10-29 15:11:33.430029: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2024-10-29 15:11:33.430073: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2024-10-29 15:11:33.430077: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:325] Ignored change_concat_input_ranges.
2024-10-29 15:11:33.432532: I tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: mnist_lenet
2024-10-29 15:11:33.434858: I tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2024-10-29 15:11:33.434922: I tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: mnist_lenet
2024-10-29 15:11:33.435824: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-10-29 15:11:33.438888: I tensorflow/compiler/mlir/

In [5]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("mnist_lenet")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_lenet.tflite"
tflite_model_file.write_bytes(tflite_model)

104472

In [9]:
# For Edge TPU.
# You must quntize your model in INT8 precision.

# Get representative data set for post-quantization.
# The representative data set prevents accuracy drop while quantization.
def representative_data_gen():
    for image in train_images[:1000]:  # Use a subset of the dataset
        # Resize the image to the input shape of your model
        image = tf.image.resize(image, (28, 28))
        image = np.expand_dims(image, axis=0)
        yield [image]

model_path = 'mnist_lenet'
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_model = converter.convert()

2024-10-29 15:13:53.028664: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2024-10-29 15:13:53.028712: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2024-10-29 15:13:53.028717: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:325] Ignored change_concat_input_ranges.
2024-10-29 15:13:53.028806: I tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: mnist_lenet
2024-10-29 15:13:53.029559: I tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2024-10-29 15:13:53.029572: I tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: mnist_lenet
2024-10-29 15:13:53.032620: I tensorflow/cc/saved_model/loader.cc:206] Restoring SavedModel bundle.
2024-10-29 15:13:53.107012: I tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: mnist_lenet
2024-10-29 15:13:53

In [11]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_lenet_quant.tflite"
tflite_model_file.write_bytes(tflite_model)

# Now, you can convert your quzntized model for Edge TPU with edgetpu_compiler.
# follow https://coral.ai/docs/edgetpu/compiler/#download

31760